# Projeto 2 - Ciência dos Dados - Estudo sobre Ifood

Nome: Lucas Hix

Nome: Andrea Mindlin Tessler

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
# Importações

import datetime
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
from random import shuffle

___
## Autenticando no  Twitter

* Conta - Lucas Hix: ***@Peng1104Oficial***
* Conta - Andrea Tessler: ***@Andrea25519779***

In [2]:
# Dados de autenticação do twitter:

def mkAPI(filename):
    # leitura do arquivo no formato JSON
    
    with open(filename) as file:
        data = json.load(file)
    
    # Configurando a biblioteca. Não modificar
    
    auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
    auth.set_access_token(data['access_token'], data['access_token_secret'])
    
    # Cria um objeto de API
    return tweepy.API(auth)

# Conta @Peng1104Oficial
api_lucas = mkAPI("lucas-auth.pass")

# Conta @Andrea25519779
api_andrea = mkAPI("andrea-auth.pass")

apis = [api_lucas, api_andrea]

___
## Etapas do projeto:

### Funções essenciais para o projeto

In [19]:
# Variaveis de suporte para a captura de Tweets
ultimo_id =-1
ultima_data = datetime.datetime.now()

# Tempo para atualização do ultimo_id (em minutos)
tempo_de_reset_do_ultimo_id = 60

# Função para a captura de Tweets
def getTweets(palavra_de_procura, quantidade_de_procuras_por_api, ultimo_id, ultima_data):
    # Lista do resultado final da captura
    resultado = []
    
    # Numero da API que ta sendo usada
    numero_api = 0
    
    # Captura o numero de tweets adicionados por API
    resultado_api = {}
    
    if ultima_data <= datetime.datetime.now():
        print("AVISO: Houve um reinicio do ultimo_id")
        ultima_data = datetime.datetime.now() + datetime.timedelta(minutes=tempo_de_reset_do_ultimo_id)
        ultimo_id = -1
    
    # Para cada API gerada
    for api in apis:
        numero_api = numero_api + 1
        
        # Contagem de captura para a API
        contagem = 0
        
        # Inicia a contagem para a API
        resultado_api[numero_api] = 0
        
        if ultimo_id < 0:
            cursor = tweepy.Cursor(api.search, q=palavra_de_procura, lang="pt", tweet_mode="extended")
        else:
            cursor = tweepy.Cursor(api.search, q=palavra_de_procura, lang="pt", tweet_mode="extended", max_id=ultimo_id)
        
        # Captura de um novo tweet
        for tweet in cursor.items(quantidade_de_procuras_por_api):
            
            # Texto do tweet capturado
            texto = tweet.full_text.lower()
                
            # Verfica se a palavra de procura se encontra no texto:
            if palavra_de_procura.lower() in texto:
                    
                # Não adiciona tweet duplicado
                if texto not in resultado:
                        
                    # Adiciona o texto ao resultado
                    resultado.append(texto)
                        
                    # Adiciona as capturas da api + 1
                    resultado_api[numero_api] = resultado_api[numero_api] + 1
                
            # Adiciona 1 captura a contagem de capturas para esta API
            contagem = contagem + 1
                
            print("API {0} de {1} - {2:.0f}% das capturas realizadas, {3} tweets validos".format(
                  numero_api, len(apis), (contagem/quantidade_de_procuras_por_api)*100, len(resultado)), end="\r")
            
            # Captura o ultimo id da captura para não duplicar pedido
            if contagem == quantidade_de_procuras_por_api:
                ultimo_id = tweet.id
            
    # Embaralhando as mensagens para reduzir um possível viés
    shuffle(resultado)
    
    for api, quantidade in resultado_api.items():
        print("A API {0} capturou {1} tweets de {2}, equivalente a {3:.2f}% do total".format(api, quantidade, len(resultado),
                                                                                         quantidade/len(resultado)*100))
    return resultado, ultimo_id, ultima_data

# Função para limpar um texto de um Tweet
def limpar_texto(texto, marca):
    # Remove Menções
    texto = re.sub("@[^ ]*", "", texto)
    # Remove links
    texto = re.sub("http(|s):\/\/[^ \n]*", "", texto)
    # Remove Pontuações
    texto = re.sub("[" + string.punctuation + "]+", "", texto)
    # Remove a marca analisada
    return re.sub(marca.lower(), "", texto)

### Escolha de uma marca e coleta das mensagens

Separação da marca a ser analisada e a quantiade de capturas a ser feitas para cada api

In [4]:
# Escolha da marca a ser analisada
marca = "Ifood"

# Esolha de quantidade por api (no caso quantidade*2)
quantidade_por_api = 500

Preparação para a nomeação dos Excels

In [5]:
# Numero de capturas ja realizadas
numero_de_capturas_realizadas = 1

# Verifica dentro do diretorio o numero de capturas realizadas
for file in os.listdir("."):
    if marca in file:
        numero_de_capturas_realizadas = numero_de_capturas_realizadas + 1

# Cria o nome do excel a ser criado
nome_base_do_arquivo = marca + "-" + str(numero_de_capturas_realizadas) 

nome_do_arquivo = nome_base_do_arquivo + ".xlsx"

Captura de dados

In [6]:
#dados_iniciais, ultimo_id, ultima_data = getTweets(marca, quantidade_por_api, ultimo_id, ultima_data)

Salva os dados capturados em um Execel

In [7]:
#tabela = pd.DataFrame({"Tweets": pd.Series(dados_iniciais), "Categoria": [""]*len(dados_iniciais)})
#tabela.to_excel(nome_do_arquivo, index = False)

#print("O Excel", nome_do_arquivo, "foi gerado!")

___
### Classificando as mensagens na coragem

Esta etapa é manual, feita pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
# Transforma os excels avaliados em um DataFrame

def criar_naive_bayes(excels, marca):
    # Cria uma tabela incial com valores inciais
    tabela_processada = pd.DataFrame({"Tweets": [""], "Categoria": [""],
                                      "Categoria atribuida automaticamente": [""]})
    
    # Junta todas as tabelas em uma e garantindo que não tenha dupliacatas na junção
    for tabela in excels:
        tabela_processada = pd.concat([tabela_processada, pd.read_excel(tabela)], sort=False).drop_duplicates(subset="Tweets")
    
    # Tabela contento todas as tabelas exluindo a tabela inicial
    tabela_processada = tabela_processada.iloc[1:]
    
    # Variavel contendo todas as categorias selecionadas
    categorias = set(tabela_processada["Categoria"])
    
    # Dicionario resultado contendo a categoria como chave e a Sereis de probabilidade como valor
    dict_resultado = {}
    
    for categoria in categorias:
        # Separa os dados da tabela com o valor selecionado uma categoria
        sub_tabela = tabela_processada[tabela_processada["Categoria"] == categoria]
        
        # Juta todas os Tweets em uma string
        mensagens = limpar_texto(sub_tabela["Tweets"].str.cat(), marca)
        
        # Cria uma series contendo todas as palavras das mensagens
        series = pd.Series(mensagens.split())
        
        # Cria uma series contendo a probabilidade de cada palavra estar na categoria selecionada
        dict_resultado[categoria] = [series.value_counts(True), (len(sub_tabela.index)/len(tabela_processada.index))]
    
    return dict_resultado

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [21]:
# Função para executar um teste dando novos tweets ultilizado um dicionanrio da naive_bayes

def executar_teste(novos_tweets, dict_de_atribuicoes, marca):
    # Lista contendo os resultados obtidos automaticamente para cada tweet novo
    resultado = []
    
    #Separa cada tweet nos novos tweets
    for tweet in novos_tweets:
        
        # Aramazena as probabilidades de cada categoria no dicionario
        probabilidades = {}
        
        # Separa cada palavra no tweet
        for palavra in limpar_texto(tweet, marca).split():
            
            # Para cada tipo de categoria
            for categoria, dados_da_probabilidade in dict_de_atribuicoes.items():
                
                # Probailidade de cada paalvra para esssa categoria
                probabilidade = dados_da_probabilidade[0]
                
                # Verifica se a palavra se encontra em uma das probabilidades da categoria
                if palavra in probabilidade:
                    
                    # Adiciona a probabilidade da palavra ao conjunto das probabilidades
                    if categoria in probabilidades:
                        probabilidades[categoria] = probabilidades[categoria] * probabilidade[palavra]
                    else:
                        probabilidades[categoria] = probabilidade[palavra] * dados_da_probabilidade[1]
        
        # Adiciona ao resultado o valor atribuido a esta mensagem
        
        # Não corespondeu a nenhuma categoria
        if len(probabilidades) == 0:
            resultado.append(-1)
        else:
            # Categoria identificado e adicionada
            resultado.append(max(probabilidades, key=probabilidades.get))
    
    return resultado

In [15]:
# Escolha da marca a ser analisada
marca = "Ifood"

# Esolha de quantidade por api (no caso quantidade*2)
quantidade_por_api = 250

In [13]:
# Cria um dicionario para o teste do naive-bayes

tabelas_avaliadas = []
tabelas_avaliadas.append("Ifood-Original.xlsx")
tabelas_avaliadas.append("Ifood-Teste-2.xlsx")

dict_naive_bayes = criar_naive_bayes(tabelas_avaliadas, marca)

In [22]:
# Gera um novo resultado

novos_tweets, ultimo_id, ultima_data = getTweets(marca, quantidade_por_api, ultimo_id, ultima_data)

resultado_dos_novos_tweets = executar_teste(novos_tweets, dict_naive_bayes, marca)

A API 1 capturou 151 tweets de 324, equivalente a 46.60% do total
A API 2 capturou 173 tweets de 324, equivalente a 53.40% do total


In [23]:
# Numero de testes ja realizados
numero_de_capturas_realizadas = 1

# Verifica dentro do diretorio o numero de testes ja realizados
for file in os.listdir("."):
    if marca in file:
        numero_de_capturas_realizadas = numero_de_capturas_realizadas + 1

# Cria o nome do excel a ser criado
nome_do_arquivo_de_teste = marca + "-Teste-" + str(numero_de_capturas_realizadas) + ".xlsx"

tabela = pd.DataFrame({"Tweets": pd.Series(novos_tweets), "Categoria": [""]*len(novos_tweets),
                       "Categoria atribuida automaticamente": resultado_dos_novos_tweets})

tabela.to_excel(nome_do_arquivo_de_teste, index=False)

print("O Excel", nome_do_arquivo_de_teste, "foi gerado!")

O Excel Ifood-Teste-4.xlsx foi gerado!


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**